In [1]:
# import libraries and modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gmaps
import scipy.stats as scistats
import random
import requests
import json
from datetime import datetime
from config import api_key
from config import gkey

In [78]:
# Kristina: Module to generate list of movie IDs

In [2]:
# mock data
dataDF = pd.DataFrame({'1970':['10671','11202','10112','10671','11202','10112'],
                       '1980':['651','11165','10364','651','11165','10364'],
                       '1990':['1685','64512','64512','10112', '9062', '11165'],
                      '2000':['1685','64512','64512','10112', '9062', '11165'],
                      '2010':['1685','64512','64512','10112', '9062', '11164']})

# Evan: Module to get list of people from cast
def top2people(movies):
    
    peopleDict = {'1970':[],'1980':[],'1990':[],'2000':[],'2010':[]}

    # loop through each movie in each column
    for column in movies.iteritems():
        for movie in column[1]:

            # request credits list for movie
            cast_url = f'https://api.themoviedb.org/3/movie/{movie}/credits?api_key={api_key}'
            data = requests.get(cast_url).json()

            # get list of cast IDs
            idList = {}
            for n,x in enumerate(data['cast']):
                idList[n] = x['cast_id']

            # determine top 2 IDs
            idDF = pd.DataFrame.from_dict(idList, orient='index', columns=['cast_id'])
            idDF.sort_values('cast_id', inplace=True)
            idDF.reset_index(drop=False, inplace=True)

            # get person ID for top 2
            person1 = data['cast'][idDF.iloc[0]['index']]['id']
            person2 = data['cast'][idDF.iloc[1]['index']]['id']

            # check for uniqueness 
            if person1 not in peopleDict[column[0]]:
                peopleDict[column[0]].append(person1)
            if person2 not in peopleDict[column[0]]:
                peopleDict[column[0]].append(person2)
                
    # convert dictionary of people IDs to DataFrame
    peopleDF = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in peopleDict.items()]))
    return peopleDF

In [3]:
mockpeopleIDs = top2people(dataDF)

In [5]:
# Aruna: Module to generate DataFrame of people with birth state and places


In [16]:
# mock data
mockPlaces = pd.DataFrame({'place':['Seattle, WA, USA',
                                   'Portland, OR, USA',
                                   'Los Angeles, CA, USA',
                                   'Kansas City, MO, USA',
                                   'Toronto, CAN']
                       })
def placesMap(places):
    
    # format location data
    searchPlaces = [loc.replace(" ", "%20") for loc in places['place']]

    # maps url
    mapUrl = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?inputtype=textquery'

    # location dictionary
    placesDict = {'lat':[], 'lng':[]}

    # iterate through places dataframe
    for city in searchPlaces:
        # pull city/state variables from the row
        search = city
        # select fields with lat/long data
        fields = 'id,geometry'
        # build query url
        url = f'{mapUrl}&input={search}&fields={fields}&key={gkey}'
        # generate json data
        data = requests.get(url).json()
        # grab lat and long data from json
        lat = data['candidates'][0]['geometry']['location']['lat']
        lng = data['candidates'][0]['geometry']['location']['lng']
        # add data to dictionary
        placesDict['lat'].append(lat)
        placesDict['lng'].append(lng)

    # location dataframe
    locationDF = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in placesDict.items()]))

    # Configure gmaps with API key
    gmaps.configure(api_key=gkey)

    # Store 'Lat' and 'Lng' into  locations 
    locations = locationDF[['lat','lng']]

    # Create a poverty Heatmap layer
    fig = gmaps.figure()

    bank_layer = gmaps.symbol_layer(
        locations, fill_color='rgba(0, 10, 0, 0.4)',
        stroke_color='rgba(0, 10, 100, 0.4)', scale=2
    )

    fig = gmaps.figure()
    fig.add_layer(bank_layer)

    # heat_layer = gmaps.heatmap_layer(locations,
    #                                  dissipating=False, max_intensity=100,
    #                                  point_radius = 1)

    # # Adjust heat_layer setting to help with heatmap dissipating on zoom
    # heat_layer.dissipating = False
    # heat_layer.max_intensity = 100
    # heat_layer.point_radius = 1

    # fig.add_layer(heat_layer)

    return fig

In [17]:
placesMap(mockPlaces)

Figure(layout=FigureLayout(height='420px'))